![](http://www.portugueslab.com/stytra/_images/dataflow_classes.svg)

# The experiment class

In [ ]:
class Experiment(QObject):
    """General class that runs an experiment.

    Parameters
    ----------
    app : QApplication()
        Application to run the Experiment QObject.
    protocol : object of :class:`Protocol <stytra.stimulation.Protocol>`
        list of protocols that can be run in this experiment session.
    directory : str
        (optional) Directory where metadata will be saved. If None, nothing
        will be
        saved (default: None).
    metadata_general: :class:`GeneralMetadata <stytra.metadata.GeneralMetadata>` object
        (optional) Class for saving general metadata about the experiment. I
        If not passed, a default GeneralMetadata object will be set.
    metadata_animal: :class:`AnimalMetadata <stytra.metadata.AnimalMetadata>` object
        (optional) Class for saving animal metadata about the experiment.
        If not passed, a default AnimalMetadata object will be set.
    calibrator : :class:`Calibrator <stytra.calibration.Calibrator>` object
        (optional) Calibrator object to calibrate the stimulus display. If
        not set, a CrossCalibrator will be used.
    asset_directory : str
        (optional) Path where asset files such as movies or images to be
        displayed can be found.
    display: dict
        (optional) Dictionary with specifications for the display. Possible
        key values are
        full_screen: bool (False)
        window_size: Tuple(Int, Int)
        framerate: target framerate, if 0, it is the highest possilbe
        gl_display : bool (False)
    rec_stim_framerate : int
        (optional) Set to record a movie of the displayed visual stimulus. It
        specifies every how many frames one will be saved (set to 1 to
        record) all displayed frames. The final movie will be saved in the
        directory in an .h5 file.
    trigger : :class:`Trigger <stytra.triggering.Trigger>` object
        (optional) Trigger class to control the beginning of the stimulation.
    offline : bool
        if stytra is used in offline analysis, stimulus is not displayed
    log_format : str
        one of "csv", "feather", "hdf5" (pytables-based) or "json"
    """

    sig_data_saved = pyqtSignal()

    def __init__(
        self,
        app=None,
        protocol=None,
        dir_save=None,
        dir_assets="",
        instance_number=-1,
        database=None,
        metadata_general=None,
        metadata_animal=None,
        loop_protocol=False,
        log_format="csv",
        scope_triggering=None,
        offline=False,
        **kwargs
    ):
        """ """
        super().__init__() # we have to do the intialisation that the QObject does, from which we inherit

        self.app = app # link to the QApplication
        self.protocol = protocol # link to the selected protocol
        self.offline = offline # offline toggle
        
        self.asset_dir = dir_assets # saving directory

        self.base_dir = dir_save
        self.database = database

        # The DataCollector collects everything with parameters and things 
        # that are explicitly saved (e.g. the stimulus running log)
        self.dc = DataCollector(
            folder_path=self.base_dir, instance_number=instance_number
        )

        self.window_main = None

        # this is an example of adding parameter data so that it will be saved (and restored)
        # together with everything
        self.metadata = GeneralMetadata(tree=self.dc)
        self.metadata_animal = AnimalMetadata(tree=self.dc)
        
        # This is done to save GUI configuration:
        self.gui_params = Parametrized(
            "gui", tree=self.dc, params=dict(geometry=Param(""), window_state=Param(""))
        )

        # another key object in Stytra, ensures running of protocols
        self.protocol_runner = ProtocolRunner(experiment=self)

        # assign signals from protocol_runner to be used externally:
        self.sig_protocol_finished = self.protocol_runner.sig_protocol_finished
        self.sig_protocol_started = self.protocol_runner.sig_protocol_started

        self.protocol_runner.sig_protocol_finished.connect(self.end_protocol)

        self.i_run = 0
        self.current_timestamp = datetime.datetime.now()

        self.gui_timer = QTimer()
        self.gui_timer.setSingleShot(False)

        self.t0 = datetime.datetime.now()

        self.animal_id = None
        self.session_id = None

# The protocol runner

In [ ]:
class ProtocolRunner(QObject):
    """Class for managing and running stimulation Protocols.

    It is thought to be
    integrated with the stytra.gui.protocol_control.ProtocolControlWidget GUI.
    
    In stytra Protocols are parameterized objects required just for generating
    a list of Stimulus objects. The engine that run this sequence of Stimuli
    is the ProtocolRunner class.
    A ProtocolRunner instance is not bound to a single Protocol object:

        - new Protocols can be set via the self.set_new_protocol() function;
        - current Protocol can be updated (e.g., after changing parameters).


    New Protocols are set by their name (a way for restoring state
    from the config.h5 file), but can also be set by passing a Protocol() class
    to the internal _set_new_protocol() method.
    Every time a Protocol is set or updated, the ProtocolRunner uses its
    get_stimulus_sequence() method to generate a new list of stimuli.
    
    For running the Protocol (i.e., going through the list of Stimulus objects
    keeping track of time), ProtocolRunner has an internal QTimer whose timeout
    calls the timestep() method, which:

        - checks elapsed time from beginning of the last stimulus;
        - if required, updates current stimulus state
        - if elapsed time has passed stimulus duration, changes current
          stimulus.


    Parameters
    ----------
    experiment : :obj:`stytra.experiment.Experiment`
        the Experiment object where directory, calibrator *et similia*
        will be found.
    dt : float
         (optional) timestep for protocol updating.
    log_print : Bool
        (optional) if True, print stimulus log.
    protocol : str
        (optional) name of protocol to be set at the beginning.


    **Signals**
    """

    sig_timestep = pyqtSignal(int)
    """Emitted at every timestep with the index of the current stimulus."""
    sig_stim_change = pyqtSignal(int)
    """Emitted every change of stimulation, with the index of the new
    stimulus."""
    sig_protocol_started = pyqtSignal()
    """Emitted when the protocol sequence starts."""
    sig_protocol_finished = pyqtSignal()
    """Emitted when the protocol sequence ends."""
    sig_protocol_updated = pyqtSignal()  # parameters changed in the protocol
    """Emitted when protocol is changed/updated"""
    sig_protocol_interrupted = pyqtSignal()

    def __init__(self, experiment=None, target_dt=0, log_print=True):
        """ """
        super().__init__()

        self.experiment = experiment
        self.target_dt = target_dt

        self.t_end = None
        self.completed = False
        self.t = 0

        self.timer = QTimer()
        self.timer.timeout.connect(self.timestep)  # connect timer to update fun
        self.timer.setSingleShot(False)

        self.protocol = experiment.protocol
        self.stimuli = []
        self.i_current_stimulus = None  # index of current stimulus
        self.current_stimulus = None  # current stimulus object
        self.past_stimuli_elapsed = None  # time elapsed in previous stimuli
        self.dynamic_log = None  # dynamic log for stimuli

        self.update_protocol()
        self.protocol.sig_param_changed.connect(self.update_protocol)

        # Log will be a list of stimuli states:
        self.log = []
        self.log_print = log_print
        self.running = False

        self.framerate_rec = FramerateRecorder()
        self.framerate_acc = FramerateAccumulator(experiment=self.experiment)

    def update_protocol(self):
        """Update current Protocol (get a new stimulus list)
        """
        self.stimuli = self.protocol._get_stimulus_list()

        self.current_stimulus = self.stimuli[0]

        # pass experiment to stimuli for calibrator and asset folders:
        for stimulus in self.stimuli:
            stimulus.initialise_external(self.experiment)

        if self.dynamic_log is None:
            self.dynamic_log = DynamicLog(self.stimuli, experiment=self.experiment)
        else:
            self.dynamic_log.update_stimuli(self.stimuli)  # new stimulus log

        self.sig_protocol_updated.emit()

    def reset(self):
        """Make the protocol ready to start again. Reset all ProtocolRunner
        and stimuli timers and elapsed times.

        """
        self.t_end = None
        self.completed = False
        self.t = 0

        for stimulus in self.stimuli:
            stimulus._started = None
            stimulus._elapsed = 0.0

        self.i_current_stimulus = 0

        if len(self.stimuli) > 0:
            self.current_stimulus = self.stimuli[0]
        else:
            self.current_stimulus = None

    def start(self):
        """Start the protocol by starting the timers.
        """
        # Updating protocol before starting has been added to include changes
        #  to the calibrator that are considered only in initializing the
        # stimulus and not while it is running (e.g., gratings). Consider
        # removing if it slows down significantly the starting event.
        self.update_protocol()
        self.log = []
        self.experiment.logger.info("{} protocol started...".format(self.protocol.name))

        self.past_stimuli_elapsed = self.experiment.t0
        self.current_stimulus.started = self.experiment.t0
        self.sig_protocol_started.emit()
        self.running = True
        self.current_stimulus.start()
        # start the timer
        self.timer.start(self.target_dt)

    def timestep(self):
        """Update displayed stimulus. This function is the core of the
        ProtocolRunner class. It is called by every timer timeout.
        At every timestep, if protocol is running:
        
            - check elapsed time from beginning of the last stimulus;
            - if required, update current stimulus state
            - if elapsed time has passed stimulus duration, change current
            stimulus.


        """
        if self.running:
            # Get total time from start in seconds:
            self.t = (datetime.datetime.now() - self.experiment.t0).total_seconds()

            # Calculate elapsed time for current stimulus:
            self.current_stimulus._elapsed = (
                datetime.datetime.now() - self.past_stimuli_elapsed
            ).total_seconds()

            # If stimulus time is over:
            if self.current_stimulus._elapsed > self.current_stimulus.duration:
                self.current_stimulus.stop()
                self.sig_stim_change.emit(self.i_current_stimulus)
                self.update_log()

                # Is this stimulus was also the last one end protocol:
                if self.i_current_stimulus >= len(self.stimuli) - 1:
                    self.completed = True
                    self.sig_protocol_finished.emit()

                else:
                    # Update the variable which keeps track when the last
                    # stimulus *should* have ended, in order to avoid
                    # drifting:

                    self.past_stimuli_elapsed += datetime.timedelta(
                        seconds=float(self.current_stimulus.duration)
                    )
                    self.i_current_stimulus += 1
                    self.current_stimulus = self.stimuli[self.i_current_stimulus]
                    self.current_stimulus.start()

            self.current_stimulus.update()  # use stimulus update function
            self.sig_timestep.emit(self.i_current_stimulus)

            # If stimulus is a constantly changing stimulus:
            if isinstance(self.current_stimulus, DynamicStimulus):
                self.sig_stim_change.emit(self.i_current_stimulus)
                self.update_dynamic_log()  # update dynamic log for stimulus

            self.framerate_rec.update_framerate()
            if self.framerate_rec.i_fps == self.framerate_rec.n_fps_frames - 1:
                self.framerate_acc.update_list(self.framerate_rec.current_framerate)

    def stop(self):
        """Stop the stimulation sequence. Update log and stop timer.
        """
        if not self.completed:  # if protocol was interrupted, update log anyway
            self.update_log()
            self.experiment.logger.info(
                "{} protocol interrupted.".format(self.protocol.name)
            )
        else:
            self.experiment.logger.info(
                "{} protocol finished.".format(self.protocol.name)
            )

        if self.running:
            self.running = False
            self.t_end = datetime.datetime.now()
            self.timer.stop()
            self.i_current_stimulus = 0
            self.t = 0
            self.sig_protocol_interrupted.emit()

    def update_log(self):
        """Append the log appending info from the last stimulus. Add to the
        stimulus info from Stimulus.get_state() start and stop times.

        """
        # Update with the data of the current stimulus:
        current_stim_dict = self.current_stimulus.get_state()
        t_stim_stop = current_stim_dict["real_time_stop"] or datetime.datetime.now()
        try:
            new_dict = dict(
                current_stim_dict,
                t_start=(
                    current_stim_dict["real_time_start"] - self.experiment.t0
                ).total_seconds(),
                t_stop=(t_stim_stop - self.experiment.t0).total_seconds(),
            )
        except TypeError as e:  # if time is None stimulus was not run
            new_dict = dict()
            logging.getLogger().info("Stimulus times incorrect, state not saved")
        self.log.append(new_dict)

    def update_dynamic_log(self):
        """
        Update a dynamic log. Called only if one is present.
        """

        self.dynamic_log.update_list(self.t, self.current_stimulus.get_dynamic_state())

    @property
    def duration(self):
        """Get total duration of the protocol in sec, calculated from stimuli
        durations.

        Returns
        -------
        float :
            protocol length in seconds.

        """
        duration = 0
        for stim in self.stimuli:
            duration += stim.duration
        return duration

    def print(self):
        """Print protocol sequence.
        """
        string = ""
        for stim in self.stimuli:
            string += "-" + stim.name

        print(string)

# Stimuli

In [ ]:
class Stimulus:
    """ Abstract class for a Stimulus.

    In stytra, a Stimulus is something that
    makes things happen at some point of an experiment.
    The Stimulus class is just a building block: successions of Stimuli
    are assembled in a meaningful order by
    :class:`Protocol.  <stytra.stimulation.Protocol>`
    objects.

    A Stimulus runs for a time defined by its duration. to do so, the
    ProtocolRunner compares at every time step the duration of the stimulus
    with the time elapsed from its beginning.
    Whenever the ProtocolRunner sets a new stimulus it calls its
    :meth:`Stimulus.start()  <Stimulus.start()>` method.
    By defining this method in subclasses, we can trigger events at
    the beginning of the stimulus (e.g., activate a Pyboard, send a TTL pulse
    or similar).
    At every successive time, until the end of the Stimulus, its
    :meth:`Stimulus.update()  <Stimulus.update()>` method is called. By
    defining this method in subclasses, we can trigger
    events throughout the length of the Stimulus time.


    Note
    ----
    Be aware that code in the :meth:`Stimulus.start()  <Stimulus.start()>`
    and :meth:`Stimulus.update()  <Stimulus.update()>`
    functions is executed within
    the Stimulus&main GUI process, therefore:

        1. Its temporal precision is limited to  **? # TODO do some check here**
        2. Slow functions would slow down the entire main process, especially if
           called at every time step.

    Stimuli have parameters that are important to be logged in the final
    metadata and parameters that are not relevant. The get_state() method
    used to generate the log saves all attributes not starting with _.


    Different stimuli categories are implemented subclassing this class, e.g.:

        - visual stimuli (children of PainterStimulus subclass);
        - ...


    Parameters
    ----------
    duration : float
         duration of the stimulus (s)
    Returns
    -------

    """

    def __init__(self, duration=0.0):
        """ """

        self.duration = duration

        self._started = None
        self._elapsed = 0.0  # time from the beginning of the stimulus
        self.name = "undefined"
        self._experiment = None
        self.real_time_start = None
        self.real_time_stop = None

    def get_state(self):
        """Returns a dictionary with stimulus features for logging.
        Ignores the properties which are private (start with _)

        Parameters
        ----------

        Returns
        -------
        dict :
            dictionary with all the current parameters of the stimulus

        """
        state_dict = dict()
        for key, value in self.__dict__.items():
            if not callable(value) and key[0] != "_":
                state_dict[key] = value
        return state_dict

    def update(self):
        """Function called by the ProtocolRunner every timestep until the Stimulus
        is over.

        Parameters
        ----------

        Returns
        -------

        """
        self.real_time_stop = datetime.datetime.now()

    def start(self):
        """Function called by the ProtocolRunner when a new stimulus is set.
        """
        self.real_time_start = datetime.datetime.now()

    def stop(self):
        """Function called by the ProtocolRunner when a new stimulus is set.
        """
        pass

    def initialise_external(self, experiment):
        """ Make a reference to the Experiment class inside the Stimulus.
        This is required to access from inside the Stimulus class to the
        Calibrator, the Pyboard, the asset directories with movies or the motor
        estimators for virtual reality.
        Also, the necessary preprocessing operations are handled here,
        such as loading images or videos.

        Parameters
        ----------
        experiment :
            the experiment object to which link the stimulus

        Returns
        -------
        type
            None

        """
        self._experiment = experiment


class DynamicStimulus(Stimulus):
    """Stimuli where parameters change during stimulation on a frame-by-frame
    base.
    It implements the recording changing parameters.

    Parameters
    ----------

    Returns
    -------

    """

    def __init__(self, *args, dynamic_parameters=None, **kwargs):
        """
        :param dynamic_parameters: A list of all parameters that are to be
                                   recorded frame by frame;
        """
        super().__init__(*args, **kwargs)
        if dynamic_parameters is None:
            self.dynamic_parameters = []
        else:
            self.dynamic_parameters = dynamic_parameters

    @property
    def dynamic_parameter_names(self):
        return [self.name + "_" + param for param in self.dynamic_parameters]

    def get_dynamic_state(self):
        """ """
        state_dict = {
            self.name + "_" + param: getattr(self, param, 0)
            for param in self.dynamic_parameters
        }
        return state_dict